In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import time
import re

In [ ]:
from sklearn.svm import SVC

In [ ]:
import sklearn.svm

In [ ]:
import xgboost as xgb

In [ ]:
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 11.1 MB/s eta 0:00:00


In [ ]:
!pip install optuna-integration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 1.2 MB/s eta 0:00:00


In [ ]:
import optuna

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [ ]:
filename = "/content/drive/My Drive/Datasets/g_2k.csv"

df = pd.read_csv(filename,
                 encoding="utf-8", encoding_errors="replace")
# replace values
df.replace({'Not Governance': 0, 'Governance': 1}, inplace=True)

# Data Preprocessing

In this section we will prepare the dataset in order to use in SVM model. Before starting I want to explain processes with 1 line explanations



*   Cleaning Everything Except Alphabetical and Numerical Characters: We'll clean unrelevant digits using regular expressions
*   Tokenizing and Lemmatizing: We'll split texts into their words and convert words to their base form (dogs=>dog)
*   Removing Stopwords: We'll remove words that have no special meaning (such as the,will,was)
*   Joining Prepared Strings: We'll join the words
*   Bag of Words Approach: We'll create a bag of word. In bag of words approach each feature shows whether the text contains the word or not.







In [ ]:
cleanedData = []

lemma = WordNetLemmatizer()
swords = stopwords.words("english")
for text in df["text"]:

    # Cleaning everything except alphabetical and numerical characters
    text = re.sub("[^a-zA-Z0-9]"," ",text)

    # Tokenizing and lemmatizing
    text = nltk.word_tokenize(text.lower())
    text = [lemma.lemmatize(word) for word in text]

    # Removing stopwords
    text = [word for word in text if word not in swords]

    # Joining
    text = " ".join(text)

    cleanedData.append(text)

In [ ]:
# check if the data is cleaned
for i in range(0,5):
    print(cleanedData[i],end="\n\n")

balance included kind donation prepared accordance b4si guideline

internal control risk management board audit committee responsible ensuring suitable internal control system prevent detect fraud error designed implemented third party service provider company also responsible reviewing effectiveness control

analysis doe take account energy mix related electricity consumption ttv portfolio issuer

change chief financial officer plc board chair position clare bousfield stepped board effect 1 october 2021 transitioned role managing director retail saving business clare continued receive salary benefit reflective new role

consideration prioritisation climate risk also built decision making governance process requirement key strategic board risk assessment paper



Now we can create our bag of words

In [ ]:
vectorizer = CountVectorizer(max_features=10000)
BOW = vectorizer.fit_transform(cleanedData)

we can split our set into train and test.

In [ ]:
x_train = list()
x_test = list()
y_train = list()
y_test = list()
for label in [0, 1]:
  x_train,x_test,y_train,y_test = train_test_split(BOW, np.asarray(df["label"]),
                                                  train_size=250,
                                                  test_size=250,
                                                  random_state=42)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(250, 4559)
(250, 4559)
(250,)
(250,)


Support Vector Machine Classifier Modeling

In [ ]:
start_time = time.time()

model = SVC()
model.fit(x_train,y_train)

end_time = time.time()
process_time = round(end_time-start_time,2)
print("Fitting SVC took {} seconds".format(process_time))

Fitting SVC took 0.02 seconds


In [ ]:
predictions = model.predict(x_test)

In [ ]:
print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))

Accuracy of model is 76.0%


# Hyperparameter tuning using Optuna

In [ ]:
clf = SVC(gamma="auto")
param_distributions = {
        "C": optuna.distributions.FloatDistribution(1e-10, 1e10, log=True),
        "degree": optuna.distributions.IntDistribution(1, 5),
    }

optuna_search = optuna.integration.OptunaSearchCV(
        clf, param_distributions, n_trials=100, timeout=600, verbose=2
    )


optuna_search.fit(x_train,y_train)

print("Best trial:")
trial = optuna_search.study_.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

<ipython-input-22-4062d26f2c32>:7: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
[I 2024-04-30 03:08:43,542] A new study created in memory with name: no-name-7cfd390a-66a4-4a52-8a81-754741ecf2e2
INFO:optuna_integration.sklearn:Searching the best hyperparameters using 250 samples...
[I 2024-04-30 03:08:43,602] Trial 0 finished with value: 0.8240000000000001 and parameters: {'C': 5106.998423617558, 'degree': 1}. Best is trial 0 with value: 0.8240000000000001.
[I 2024-04-30 03:08:43,656] Trial 1 finished with value: 0.8240000000000001 and parameters: {'C': 72537.74955294652, 'degree': 4}. Best is trial 0 with value: 0.8240000000000001.
[I 2024-04-30 03:08:43,702] Trial 2 finished with value: 0.772 and parameters: {'C': 5.945285082062503, 'degree': 4}. Best is trial 0 with value: 0.8240000000000001.
[I 2024-04-30 03:08:43,759] Trial 3 finished with value: 0.824000000

Best trial:
  Value:  0.8360000000000001
  Params: 
    C: 325.07602123357964
    degree: 4


In [ ]:
trial.params

{'C': 325.07602123357964, 'degree': 4}

In [ ]:
start_time = time.time()

model = SVC(gamma="auto", degree=trial.params['degree'], C=trial.params['C'])
model.fit(x_train,y_train)

end_time = time.time()
process_time = round(end_time-start_time,2)
print("Fitting SVC took {} seconds".format(process_time))

Fitting SVC took 0.02 seconds


In [ ]:
trial.params['degree']

4

In [ ]:
predictions = model.predict(x_test)

In [ ]:
print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))

Accuracy of model is 77.2%


In [ ]:
def evaluate(y_true, y_pred):
    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [ ]:
evaluate(y_true=y_test, y_pred=predictions)

Accuracy: 0.772
Accuracy for label 0: 0.893
Accuracy for label 1: 0.413

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.89      0.85       187
           1       0.57      0.41      0.48        63

    accuracy                           0.77       250
   macro avg       0.69      0.65      0.67       250
weighted avg       0.75      0.77      0.76       250


Confusion Matrix:
[[167  20]
 [ 37  26]]


# build model for XGBoost

In [ ]:
start_time = time.time()

xgb_model = xgb.XGBClassifier()
xgb_model.fit(x_train,y_train)

end_time = time.time()
process_time = round(end_time-start_time,2)
print("Fitting XGBoost took {} seconds".format(process_time))

Fitting XGBoost took 1.03 seconds


In [ ]:
predictions = model.predict(x_test)

In [ ]:
print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))

Accuracy of model is 77.2%


# Hyperparameter tuning using Optuna

In [ ]:
def objective(trial):
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dvalid = xgb.DMatrix(x_test, label=y_test)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_test,predictions)
    return accuracy

In [ ]:
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize"
    )
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2024-04-30 03:08:50,889] A new study created in memory with name: no-name-bf170eee-9242-456e-933c-ddcaa3c0b719


[0]	validation-auc:0.50000
[1]	validation-auc:0.50000
[2]	validation-auc:0.50000
[3]	validation-auc:0.50000
[4]	validation-auc:0.50000
[5]	validation-auc:0.50000
[6]	validation-auc:0.50000
[7]	validation-auc:0.50000
[8]	validation-auc:0.50000
[9]	validation-auc:0.50000


[I 2024-04-30 03:08:51,387] Trial 0 finished with value: 0.772 and parameters: {'booster': 'gblinear', 'lambda': 0.0007990125451767318, 'alpha': 0.616465210595442}. Best is trial 0 with value: 0.772.


[0]	validation-auc:0.80872
[1]	validation-auc:0.80872
[2]	validation-auc:0.80872
[3]	validation-auc:0.80150
[4]	validation-auc:0.80702
[5]	validation-auc:0.80634
[6]	validation-auc:0.80566
[7]	validation-auc:0.80643
[8]	validation-auc:0.80634
[9]	validation-auc:0.80566


[I 2024-04-30 03:08:51,720] Trial 1 finished with value: 0.772 and parameters: {'booster': 'gbtree', 'lambda': 0.004225324677069771, 'alpha': 0.13902716639794016, 'max_depth': 8, 'eta': 0.014452122661195197, 'gamma': 0.4488658235595789, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.772.


...

[0]	validation-auc:0.74960
[1]	validation-auc:0.77557
[2]	validation-auc:0.78958
[3]	validation-auc:0.79586
[4]	validation-auc:0.80010


[I 2024-04-30 03:09:01,467] Trial 98 pruned. Trial was pruned at iteration 5.


[0]	validation-auc:0.82964
[1]	validation-auc:0.81950
[2]	validation-auc:0.81733
[3]	validation-auc:0.81767
[4]	validation-auc:0.83630
[5]	validation-auc:0.85112
[6]	validation-auc:0.84925
[7]	validation-auc:0.85685
[8]	validation-auc:0.86075
[9]	validation-auc:0.86410


[I 2024-04-30 03:09:01,579] Trial 99 finished with value: 0.772 and parameters: {'booster': 'gbtree', 'lambda': 0.0018448647108756885, 'alpha': 0.015197810231905876, 'max_depth': 9, 'eta': 0.0809554197879324, 'gamma': 0.0007425061330350752, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.772.


FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.772], datetime_start=datetime.datetime(2024, 4, 30, 3, 8, 50, 890717), datetime_complete=datetime.datetime(2024, 4, 30, 3, 8, 51, 387358), params={'booster': 'gblinear', 'lambda': 0.0007990125451767318, 'alpha': 0.616465210595442}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.5, 1: 0.5, 2: 0.5, 3: 0.5, 4: 0.5, 5: 0.5, 6: 0.5, 7: 0.5, 8: 0.5, 9: 0.5}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': FloatDistribution(high=1.0, log=True, low=1e-08, step=None), 'alpha': FloatDistribution(high=1.0, log=True, low=1e-08, step=None)}, trial_id=0, value=None)


In [ ]:
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

Best trial:
  Value:  0.772
  Params: 
    booster: gblinear
    lambda: 0.0007990125451767318
    alpha: 0.616465210595442


In [ ]:
start_time = time.time()

xgb_model = xgb.XGBClassifier(**trial.params)
xgb_model.fit(x_train,y_train)

end_time = time.time()
process_time = round(end_time-start_time,2)
print("Fitting XGBoost took {} seconds".format(process_time))

Fitting XGBoost took 0.01 seconds


In [ ]:
predictions = model.predict(x_test)

In [ ]:
print("Accuracy of model is {}%".format(accuracy_score(y_test,predictions) * 100))

Accuracy of model is 77.2%


In [ ]:
evaluate(y_true=y_test, y_pred=predictions)

Accuracy: 0.772
Accuracy for label 0: 0.893
Accuracy for label 1: 0.413

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.89      0.85       187
           1       0.57      0.41      0.48        63

    accuracy                           0.77       250
   macro avg       0.69      0.65      0.67       250
weighted avg       0.75      0.77      0.76       250


Confusion Matrix:
[[167  20]
 [ 37  26]]
